In [ ]:
# https://github.com/taehojo/deeplearning/blob/master/ch18.ipynb
# 아마 위 주소를 하지 않을까?

In [ ]:
# cnn은 이미지의 특징을 추출하여 이미지를 분류하는 방식임. 근데 텍스트 분류에서도 사용이 가능함.
# 필터를 만드는 과정이 사실 학습하는 것이라고 볼 수 있음(원래는 랜덤한 값으로 시작하니까)
# 자연어처리에 사용하는 CNN은 Conv1D를 이용하게됨(이미지는 2D, 동영상은 3D임)
# https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE09474867
# 위 주소와 같이 cnn-lstm을 이용한 문맥 반영 및 감정 분류가 가능함

https://thebook.io/080324/0296/ 해당 주소 참고

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [6]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)
print(X_train.shape)
# 학습데이터만 25000건이 있는데 그럼 단어는 수만가지 이상 있을 것임. 
# 이중 빈도가 높은 단어 5000건만 이용하겠다는 것임
# 타겟 데이터인 긍/부정을 y_train, y_test에 담음

(25000,)


In [11]:
len(X_train[0]) # 218
y_train[0] # 1

1

In [ ]:
# ex) 그 영화 너무 감동적이었어요. 주인공 정말 멋져요. 왕 추천!!!
# 1) 텍스트 전처리(구문분석, 형태소 분리, 불필요한 형태소 제거, 동음이의어, 다의어 처리)
# 은행 계좌를 만들려고 합니다
#           생성하려고
#             틀려고
#            가지려고
#      구좌를 만들려고
#        ...   ....

# 영화 너무 웃겨서 배 아팠어요 => 50
# 영화 보면서 배를 먹었어요 => 15
# 전에 봤던 것보다 2배 이상 더 잼있었어요 => 2100
# 위와 같이 '배'라는 단어를 서로 다르게 구분할 수 있는 문맥인지 능력이 있어야함

# 위와 같이 문맥을 반영하는 것이 오랫동안 해결되지 않았음

# transformer를 부모로 둔 모델 gpt, bert는 문맥을 어느정도 반영을 할 수 있게 됨

In [12]:
X_train = sequence.pad_sequences(X_train, maxlen=500) # 모든 문장의 길이를 500으로 설정
X_test = sequence.pad_sequences(X_test, maxlen=500)

In [13]:
model = Sequential()
model.add(Embedding(5000, 100)) # 임베딩 -> 차원축소
model.add(Dropout(0.5)) # 50% 사용하도록 설정
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
# 64는 핉터 개수임(즉 특징 64개를 찾겠다는 의미임), 5는 커널사이즈(2D에서는 2x2이런식인데 여기서는 1D이므로 숫자 하나만 있으면 됨)
# same은 크기를 같게 valid는 크기를 점점 줄이는 것(일반적으로 valid를 이용함)
# 예시로 그 영화 너무 재밌어요 라는 문장이면 '그 영화 '까지 5커널사이즈에 들어가므로 연산을 하고 연산 결과를 풀링을 적용
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
# return_sequences=False가 생략됨
model.add(Dense(1))
model.add(Activation('sigmoid'))
# 긍/부정 분류
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         500000    
                                                                 
 dropout (Dropout)           (None, None, 100)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 64)          32064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 64)         0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 55)                26400     
                                                                 
 dense (Dense)               (None, 1)                 56        
                                                        

In [14]:
# 모델의 실행 옵션을 정합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 학습의 조기 중단을 설정합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)
# 모델을 실행합니다.
history = model.fit(X_train, y_train, batch_size=40, epochs=100, validation_split=0.25, callbacks=[early_stopping_callback])
# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

Epoch 1/100
469/469 [==============================] - 28s 14ms/step - loss: 0.4088 - accuracy: 0.7980 - val_loss: 0.2806 - val_accuracy: 0.8858
Epoch 2/100
469/469 [==============================] - 6s 13ms/step - loss: 0.2342 - accuracy: 0.9099 - val_loss: 0.2716 - val_accuracy: 0.8885
Epoch 3/100
469/469 [==============================] - 6s 12ms/step - loss: 0.1848 - accuracy: 0.9314 - val_loss: 0.2708 - val_accuracy: 0.8912
Epoch 4/100
469/469 [==============================] - 6s 13ms/step - loss: 0.1453 - accuracy: 0.9459 - val_loss: 0.2846 - val_accuracy: 0.8886
Epoch 5/100
469/469 [==============================] - 6s 13ms/step - loss: 0.1200 - accuracy: 0.9572 - val_loss: 0.3884 - val_accuracy: 0.8686
Epoch 6/100
782/782 [==============================] - 4s 5ms/step - loss: 0.3785 - accuracy: 0.8775

 Test Accuracy: 0.8775


In [ ]:
# 강사님은 인공지능 연구만 15년넘게 하심에도 인공지능에 대해 아직 모르는 것이 많다고 하심

In [ ]:
# 이미지넷 대회는 계속 이미지 데이터셋이 바뀜. 그래서 매번 같은 모델이 우승할 수 없는 구조인듯함

In [ ]:
# transfer learning(전이학습) = fine tuning
# yolo, GANs를 다음주에 진행할 것임
# 전이학습이란? 알고있는 지식이 있을 때 이 지식을 타인에게 전달해주고 
# 전달받은 대상은 그 지식을 기반으로 추가적인 학습을 진행하는 것이라고 보면 됨

# 예를 들어 한국어를 공부한 사람이 중국어를 배우는 상황과 언어를 모르는 사람이 중국어를 공부하는 것과 속도를 비교하면 
# 한국어를 공부하고 중국어를 공부하는 것이 훨씬 빠름

In [19]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
last_layer = base_model.get_layer('block5_pool')
last_output = last_layer.output
x = Flatten()(last_output)
x = Dense(2, activation='softmax', name = 'softmax')(x)
new_model = Model(inputs=base_model.input, outputs=x)
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [17]:
# 강아지/고양이를 분류하는 모델을 개발함
# 근데 소/돼지도 분류하는 것을 추가하고 싶음
# 이런 때 강아지/고양이를 분류하는 모델의 가중치를 가져와서 적용
# 그럼 원래 강아지와 고양이만 분류하는 것이 가능했는데 추가적인 항목을 통해서 분류하는 방법을 더 익히게 됨
# GPT는 모델이름임. BERT, LLAMA도 모델이름.
# 여기에 내가 가진 데이터를 추가학습 진행(파인튜닝은 전이학습에 포함되는 것임)

# 사람에서 자동차는? 코끼리? 의자?
# 이런 것들도 저수준 학습단계에서는 선을 추출하는 단계로 시작하게 되는데 이런 것들은 다른 데이터도 동일하게 진행되므로
# 동일하게 모델을 적용할 수 있으며 고수준 레벨일 경우 다른 모델에서 적용하기가 어려워짐

# 결국 전이학습을 하기 위해서는 저수준에 해당하는 것을 재사용하고 고수준은 추가학습으로 정확도를 올리는 것임

# 전이학습에는 3가지 시나리오가 있음


In [21]:
# 담주 월 화욜 욜로 나머지는 자연어처리